#  Investigating Fairness in Data-Driven Allocation of Public Resources

---



This notebook demonstrates, through a real-world use case, how IBM's [AIF360 toolkit](https://github.com/Trusted-AI/AIF360) can be used to evaluate the fairness of an algorithm for allocating public resources. The data used was generated specifically for this workshop and therefore does not allow any conclusions to be drawn about individuals. Further assumptions that were made serve as a simplified representation of a complex problem and do not reflect the values of the presenters. 

First, a logistic regression is used to predict the risk of long-term unemployment of job seekers. If a predefined threshold is exceeded, i.e. if the probability of LTU is high according to the algorithm, the person will receive supportive measures from the public employment service.

As a protected attribute we consider the variable gender, which is binary coded here. Therefore, in a second step, we will investigate whether the prediction for LTU differs between men and women. To do so, we evaluate the Statistical Parity Difference.

Finally, we demonstrate techniques to mitigate existing biases. Here we apply reweighing, a method that assigns weights to individual instances in the training data based on frequency counts of the protected attribute and the actual outcome before classification.

## Install aif360

In [ ]:
%pip install aif360

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import necessary packages

In [ ]:
%matplotlib inline

import sys
sys.path.append("../")

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
from collections import OrderedDict, defaultdict
from IPython.core.display import Markdown
from IPython.display import display

from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

from aif360.datasets import BinaryLabelDataset

# Classifier
from sklearn.linear_model import LogisticRegression

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# Bias mitigation technique
from aif360.algorithms.preprocessing import Reweighing

# Set seed
seed = 1
np.random.seed(seed)

## Helper functions

In this section, helper functions are defined that can be used on different datasets or with varying thresholds. 

In [ ]:
def get_dataset_with_scores(dataset, model):
    """Returns a dataset with the scores predicted by the given model.

    Args:
        dataset (BinaryLabelDataset): The dataset to be predicted.
        model(e.g. LogisticRegression): Classification model.
        """
    dataset_pred = dataset.copy(deepcopy=True)
    pos_ind = np.where(model.classes_ == dataset.favorable_label)[0][0]
    y_pred_prob = model.predict_proba(dataset.features)[:, pos_ind]
    dataset_pred.scores = y_pred_prob.reshape(-1, 1)
    
    return dataset_pred

In [ ]:
def get_label_at_threshold(dataset_pred, threshold):
    """Returns a dataset with new labels according to the specified threshold. 
    
    The instances whose score is greater than the specified threshold are 
    assigned the favorable_label given in the BinaryLabelDataset.

    Args:
        dataset_pred (BinaryLabelDataset): BinaryLabelDataset with predicted scores.
        threshold (float): Threshold to assign labels from given scores.
        """
    dataset_pred_thresh = dataset_pred.copy(deepcopy=True)
    fav_inds = dataset_pred_thresh.scores > threshold
    dataset_pred_thresh.labels[fav_inds] = dataset_pred_thresh.favorable_label
    dataset_pred_thresh.labels[~fav_inds] = dataset_pred_thresh.unfavorable_label

    return dataset_pred_thresh

In [ ]:
def get_dataset_with_predictions(dataset, model, threshold):
    """Returns a dataset with new scores predicted by the given model and new 
    labels corresponding to the given threshold.

    Args:
        dataset (BinaryLabelDataset): The dataset to be used for prediction.
        model (e.g. LogisticRegression): Classification model.
        threshold (float): Threshold to assign labels from scores.
        """
    dataset_pred = get_dataset_with_scores(dataset, model)
    return get_label_at_threshold(dataset_pred, threshold)

In [ ]:
def plot_metric_arr(ax, metric_dict, metric_transf_dict=None, thresh=None, ylim=None, hline_at=None):
    """Plots the given metric_dict on the axis specified.

    Args:
        ax (Axes): The axis to be plottet on.
        metric_dict (Dict): Dict with metrics as keys and list of metric value 
        at different thresholds as values.
        metric_transf_dict (Dict, optional): Dict with metrics as keys and list of metric value mitigation at different thresholds as values.
        thresh (float, optional): Threshold to be indicated by a dotted line in the plot.
        ylim (tuple, optional): Set ylim for given axis.
        hline_at (float, optional): If not None, hline will be highlighted at given value.
        """
    for metric, metric_val in metric_dict.items():
        ax.plot(thresh_arr, metric_val, label=metric)
        if metric_transf_dict:
            ax.plot(thresh_arr, metric_transf_dict[metric], label=f'{metric}_transf', linestyle='dashed', color=ax.get_lines()[-1].get_c())
        ax.set_xlabel('Classification Threshold', fontsize=16, fontweight='bold')
        ax.xaxis.set_tick_params(labelsize=14)
        ax.yaxis.set_tick_params(labelsize=14)
        ax.grid(True)

        if hline_at is not None:
            ax.axhline(hline_at, linewidth=2, color='k')

        if thresh:
            thresh_ind = np.where(np.round(thresh_arr, 2) == thresh)[0][0]
            ax.axvline(np.array(thresh_arr)[thresh_ind], color='k', linestyle=':')

    if ylim:
        plt.setp([ax], ylim=ylim)
    plt.legend()

In [ ]:
def plot_metrics_over_thresholds(performance_metric_dict, fairness_metric_dict, thresh_arr, threshold):
    """Plots all metrics in the given metric array over all thresholds in the 
    given threshold array.

    Args:
        performance_metric_dict, fairness_metric_dict (Dict): Dict with metrics as keys and list of metric value 
        at different thresholds as values.
        thresh_arr (Array): Array with different thresholds.
        threshold (float, optional): Threshold to be indicated by a dotted line in the plot.
        """

    fig = plt.figure(figsize=(14, 12))
    ax1 = plt.subplot(2, 2, 1)
    plot_metric_arr(ax1, performance_metric_dict, thresh=threshold, ylim=(0 ,1))

    min_val = min([v for val_list in fairness_metric_dict.values() for v in val_list])
    max_val = max([v for val_list in fairness_metric_dict.values() for v in val_list])
    custom_ylim = (round(min_val, 1)-0.1, round(max_val+0.1, 1))

    ax2 = plt.subplot(2, 2, 2)
    plot_metric_arr(ax2, fairness_metric_dict, thresh=threshold, ylim=custom_ylim, hline_at=0)

    fig.tight_layout()

In [ ]:
def get_classification_metrics(dataset_true, dataset_pred, f1=True):
    """Returns classification metrics for two BinaryLabelDatasets. The first 
    dataset is the true one, the second includes the predicted scores.

    Args:
        dataset_true (BinaryLabelDataset): Dataset with true labels.
        dataset_pred (BinaryLabelDataset): Dataset with predicted scores.
        f1 (bool, optional): If True, calculate F1 score for classification.
        """
    performance_metrics = OrderedDict()
    fairness_metrics = OrderedDict()

    classified_metric = ClassificationMetric(dataset_true, dataset_pred,
                                                  unprivileged_groups=unprivileged_groups,
                                                  privileged_groups=privileged_groups)
    
    if f1:
        if classified_metric.precision() == 0.0 and classified_metric.recall() == 0.0:
            return None
        else:
            f1_score = 2*((classified_metric.precision()*classified_metric.recall())/(classified_metric.precision() + classified_metric.recall()))
            performance_metrics["F1 score"] = f1_score
    
    accuracy = classified_metric.accuracy()           
    spd = classified_metric.statistical_parity_difference()
    eod = classified_metric.equal_opportunity_difference()

    performance_metrics["Accuracy"] = accuracy
    fairness_metrics["Statistical parity difference"] = spd
    fairness_metrics["Equal opportunity difference"] = eod

    return performance_metrics, fairness_metrics

In [ ]:
def get_fairness_over_thresholds(dataset, model, thresh_arr, unprivileged_groups, privileged_groups, threshold=None, dataset_with_scores=None):
    """Plots fairness metrics over thresholds in the specified thresh_arr.

    Args:
        dataset (BinaryLabelDataset): Dataset with true labels.
        model (LogisticRegression): Classification model.
        thresh_arr (Array): Array with different thresholds.
        unprivileged_groups (list(dict)): Unprivileged groups used as input for ClassificationMetric. For details, see AIF360 documentation.
        privileged_groups (list(dict)): Privileged groups used as input for ClassificationMetric. For details, see AIF360 documentation.
        threshold (float, optional): 
        """
    if not dataset_with_scores:
        dataset_with_scores = get_dataset_with_scores(dataset, model)
    performance_metric_dict = defaultdict(list)
    fairness_metric_dict = defaultdict(list)

    for i, thresh in enumerate(thresh_arr):
        dataset_pred = get_label_at_threshold(dataset_with_scores, thresh)
        performance_metrics, fairness_metrics = get_classification_metrics(dataset, dataset_pred)

        for (metric, value) in performance_metrics.items():
            performance_metric_dict[metric].append(value)
        
        for (metric, value) in fairness_metrics.items():
            fairness_metric_dict[metric].append(value)
    
    plot_metrics_over_thresholds(performance_metric_dict, fairness_metric_dict, thresh_arr, threshold)

In [ ]:
def plot_metrics_transf_over_thresholds(perf_metric_dict, perf_metric_transf_dict, fair_metric_dict, fair_metric_transf_dict, thresh_arr, threshold):
    """Plots all metrics in the given metric dict and corresponding metrics for 
    mitigated dataset (metric_transf_dict) over all thresholds in the given threshold array.

    Args:
        perf_metric_dict, fair_metric_dict (Dict): Dict with metrics as keys and list of metric value at different thresholds as values.
        perf_metric_trasnf_dict, fair_metric_transf_dict (Dict): Dict with metrics as keys and list of metric value mitigation at different thresholds as values.
        thresh_arr (Array): Array with different thresholds.
        threshold (float, optional): Threshold to be indicated by a dotted line in the plot.
        """

    fig = plt.figure(figsize=(14, 12))

    ax1 = plt.subplot(2, 2, 1)
    plot_metric_arr(ax1, perf_metric_dict, perf_metric_transf_dict, thresh=threshold, ylim=(0, 1))

    fair_metric_values = list(fair_metric_dict.values()) + list(fair_metric_transf_dict.values())
    min_val = min([v for val_list in fair_metric_values for v in val_list])
    max_val = max([v for val_list in fair_metric_values for v in val_list])
    custom_ylim = (round(min_val, 1)-0.1, round(max_val+0.1, 1))
    ax2 = plt.subplot(2, 2, 2)
    plot_metric_arr(ax2, fair_metric_dict, fair_metric_transf_dict, thresh=threshold, ylim=custom_ylim, hline_at=0)

    fig.tight_layout()

In [ ]:
def get_fairness_mitigation_over_thresholds(dataset_true, dataset_pred, dataset_transf_pred, thresh_arr, unprivileged_groups, privileged_groups, threshold=None):
    """Plots fairness metrics before and after mitigation over thresholds in the specified thresh_arr.

    Args:
        dataset_true (BinaryLabelDataset): Dataset with true labels.
        dataset_pred (BinaryLabelDataset): Dataset with predicted scores.
        dataset_transf_pred (BinaryLabelDataset): Dataset with predicted scores after mitigation.
        thresh_arr (Array): Array with different thresholds.
        unprivileged_groups (list(dict)): Unprivileged groups used as input for ClassificationMetric. For details, see AIF360 documentation.
        privileged_groups (list(dict)): Privileged groups used as input for ClassificationMetric. For details, see AIF360 documentation.
        threshold (float, optional): 
        """
    perf_metric_dict = defaultdict(list)
    perf_metric_transf_dict = defaultdict(list)
    fair_metric_dict = defaultdict(list)
    fair_metric_transf_dict = defaultdict(list)

    for i, thresh in enumerate(thresh_arr):
        dataset_pred_thresh = get_label_at_threshold(dataset_pred, thresh)
        dataset_transf_pred_thresh = get_label_at_threshold(dataset_transf_pred, thresh)

        performance_metrics, fairness_metrics = get_classification_metrics(dataset_true, dataset_pred_thresh)

        for (metric, value) in performance_metrics.items():
            perf_metric_dict[metric].append(value)
        
        for (metric, value) in fairness_metrics.items():
            fair_metric_dict[metric].append(value)

        performance_metrics_transf, fairness_metrics_transf = get_classification_metrics(dataset_true, dataset_transf_pred_thresh)
        
        for (metric, value) in performance_metrics_transf.items():
            perf_metric_transf_dict[metric].append(value)
        
        for (metric, value) in fairness_metrics_transf.items():
            fair_metric_transf_dict[metric].append(value)
    
    print("The solid line indicates values before bias mitigation.\nThe dashed line indicates values after mitigation.")
    plot_metrics_transf_over_thresholds(perf_metric_dict, perf_metric_transf_dict, fair_metric_dict, fair_metric_transf_dict, thresh_arr, threshold)

In [ ]:
def show_variable_distr(df_var, x, hue=None, legend=None):
    """Plots distribution of variables in the given dataset.

    Args:
        df_var (DataFrame): Pandas DataFrame object.
        x (str): Attribute to be plotted on the x-axis.
        hue (str): Attribute to separate data on x-axis.
        legend (list(str), optional): List of strings to use as legend values.
        """
    df_distr = df_var[[x]] if not hue else df_orig.groupby(x)[hue]
    df_distr = df_distr.value_counts(normalize=True)
    df_distr = df_distr.mul(100)
    df_distr = df_distr.rename('percent').reset_index()

    g = sns.catplot(x=x,y='percent',hue=hue,kind='bar', data=df_distr, legend=False if legend else True)
    g.ax.set_ylim(0,100)
    if legend:
        ax_legend = g.ax.legend(title=hue)
        ax_legend.texts[0].set_text(legend[0])
        ax_legend.texts[1].set_text(legend[1])

    for p in g.ax.patches:
        g.ax.annotate(f'\n{round(p.get_height(),2)}%',
                    (p.get_x() + (0.38 if not hue else 0.2),
                    p.get_height()),
                    ha='center',
                    va='top', color='black',
                    size=12)

In [ ]:
def show_dataset_distributions(dataset, dataset_pred=None):
    """Print distribution of target  and sens_attr in the given dataset and the dataset_pred if specified.

    Args:
        dataset (BinaryLabelDataset): Dataset to show distributions for. 
        dataset_pred (BinaryLabelDataset, optional): If not None, show distribution for this dataset.
        """
    df = dataset.convert_to_dataframe()[0]
    display(Markdown(f"#### Dataset {target} distribution"))
    print(df.value_counts(target, normalize=True).round(4))
    display(Markdown(f"#### Dataset {sens_attr} distribution"))
    print(df.value_counts(sens_attr, normalize=True).round(4))
    display(Markdown(f"#### Dataset {target} x {sens_attr} distribution"))
    print(pd.crosstab(df[target],df[sens_attr], normalize=True).round(4))

    if dataset_pred:
        df_pred = dataset_pred.convert_to_dataframe()[0]
        target_pred = f'{target}_pred'
        df_pred = df_pred.rename(columns={target: target_pred})
        display(Markdown(f"#### Dataset {target} predicted distribution"))
        print(df_pred.value_counts(target_pred, normalize=True).round(4))
        display(Markdown(f"#### Dataset {target} x {sens_attr} predicted distribution"))
        print(pd.crosstab(df_pred[target_pred],df_pred[sens_attr], normalize=True).round(4))

In [ ]:
def show_conf_matrix(dataset_true, dataset_pred, per_group=True):
    """Print confusion matrix for given datasets as markdown table. If per_group is True, confusion matrix will be shown for each subgroup of the sensitive attribute.

    Args:
        dataset, dataset_pred (BinaryLabelDataset): Datasets to show confusion matrix for. 
        per_group (bool, optional): If yes, show confusion matrix for each subgroup of the sensitive attribute.
        """

    conf_matrix_dict = OrderedDict()

    if per_group:
        df_true = dataset_true.convert_to_dataframe()[0]
        df_pred = dataset_pred.convert_to_dataframe()[0]
        for value in df_true[sens_attr].unique():
            df_true_group = df_true[df_true[sens_attr] == value]
            df_pred_group = df_pred[df_pred[sens_attr] == value]
            conf_matrix = confusion_matrix(df_true_group[target], df_pred_group[target], normalize='all')
            conf_matrix_dict[value] = conf_matrix
    else:
        conf_matrix_dict[0] = confusion_matrix(dataset_true.labels, dataset_pred.labels, normalize='all')
    
    for (k, conf_matrix) in conf_matrix_dict.items():
        if per_group:
            display(Markdown(f"#### Confusion matrix for {sens_attr} = {k}"))
        tn, fp, fn, tp = conf_matrix.ravel()
        df_metrics = pd.DataFrame({'':['LTU', 'Non LTU'],
                    'Pred LTU':[tp, fp],
                    'Pred non LTU':[fn, tn]})
        
        print(df_metrics.round(4).to_markdown(index=False))

In [ ]:
def show_dataset_metrics(dataset):
    """Show dataset metrics for given dataset.

    Args:
        dataset (BinaryLabelDataset): Dataset to get dataset metrics from. 
        """
    show_dataset_distributions(dataset)

    dataset_metric = BinaryLabelDatasetMetric(
        dataset,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
    
    metric_dict = OrderedDict()
    metric_dict["Statistical parity difference"] = dataset_metric.statistical_parity_difference()

    display(Markdown(f"#### Dataset metrics"))
    for k in metric_dict:
            print(f"{k} = {metric_dict[k]:.4f}")

In [ ]:
def show_classification_metrics(dataset_true, dataset_pred, f1=True):
    """Show classification metrics for given datasets.

    Args:
        dataset, dataset_pred (BinaryLabelDataset): Dataset to get classification metrics from. 
        f1 (bool, optional): If True, get F1 score.
        """
    show_dataset_distributions(dataset_true, dataset_pred)
    show_conf_matrix(dataset_true, dataset_pred) 
   
    performance_metrics_dict, fairness_metrics_dict = get_classification_metrics(dataset_true, dataset_pred, f1)
    classification_metrics_dict = dict(performance_metrics_dict, **fairness_metrics_dict)

    display(Markdown(f"#### Classification metrics"))
    for k in classification_metrics_dict:
        print(f"{k} = {classification_metrics_dict[k]:.4f}")

# Data generation

In the data generation phase, the data used for the prediction task is gathered, selected and, if necessary, labeled. The labels are later used to assess the "true outcome" of the variable to be predicted. In our case, this is the probability of long term unemployment (LTU).

## Loading the data

In this cell, a dataset from [GitHub](https://github.com/), a code sharing platform and version control tool, is loaded into our notebook. Then, we output the head (first five rows) of the dataframe (like a data table) to get a first impression of the data set at hand.

In [ ]:
import pandas as pd
df_orig = pd.read_csv('https://raw.githubusercontent.com/achterhe/fair-ml-goethe/main/daten.csv', index_col=0)
df_orig.head()

# Data preparation and analysis

In this step, the data is first prepared to be later used as input for the classification model. The sensitive attribute is postulated, variable distributions are explored and model decisions, like the classification threshold, are specified.

In [ ]:
# Set relevant variables

sens_attr = 'gender'

privileged_groups = [{sens_attr: 0}] # Male
unprivileged_groups = [{sens_attr: 1}] # Female

target = 'ltue'

threshold = 0.5

## Analyze the data

To gain insight into the available data, we plot the distribution of the target variable and the sensitive attribute.

In [ ]:
display(Markdown("#### Distribution of target variable"))
show_variable_distr(df_orig, target)

In [ ]:
display(Markdown("#### Distribution of sensitive attribute"))
show_variable_distr(df_orig, sens_attr)

In [ ]:
display(Markdown("#### Distribution of target per sensitive attribute"))
show_variable_distr(df_orig, target, hue=sens_attr, legend=['M', 'F'])

## Train-Test split

In the next step, we convert our data into a BinaryLabelDataset, a dataset type that is required for further processing of the data with the AIF360 toolkit. At this point, you may wonder about the "favorable_label" parameter, which we have set to 1 for this use case. Remember that ltue=1 means that the person is actually LTU. So is this really the favorable outcome? A look at the [documentation](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.BinaryLabelDataset.html#aif360.datasets.BinaryLabelDataset) of the BinaryLabelDataset class shows that "favorable" is used here in the same way as "positive outcome", which is more common in ML and denotes predictions that are classified as 1. However, this does not explicitly indicate whether this prediction is the desired one, but depends on interpretation. Therefore, we criticize the use of "favorable" and "unfavorable" at this point.

In [ ]:
# Create a BinaryLabelDataset

dataset_orig = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_orig,
    label_names=[target],
    protected_attribute_names=[sens_attr]
)

Next, we split the data into a training set and a test set. A validation set is not considered for this use case. 
With a split parameter of 0.7, 70% of the instances will belong to the training data set, while 30% will be used for the test data set. By splitting the data, we are able to evaluate the performance of our model. The model learns relationships between the features in the training dataset and the true label. Next, the trained model is used to predict the results of the test data set. To evaluate the model, we then compare the predictions to the expected label we have in the test data.

In [ ]:
# Split the data into training, validation and test set

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True, seed=seed)

After the split, we can explore the training data set again

In [ ]:
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes,
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

## Learning a classifier

Now we will train a classifier with logistic regression on the test data and then make predictions on the test data. Performance evaluation can be done using the confusion matrix and the classification report.

In [ ]:
lmod = LogisticRegression(random_state=seed, solver="liblinear")
lmod.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())

In [ ]:
dataset_orig_test_pred = get_dataset_with_predictions(dataset_orig_test, lmod, threshold)

In [ ]:
display(Markdown(f"## Confustion matrix"))
show_conf_matrix(dataset_orig_test, dataset_orig_test_pred, per_group=False)

In [ ]:
display(Markdown(f"## Classification report"))
print(classification_report(dataset_orig_test.labels, dataset_orig_test_pred.labels))

## Calculate fairness metrics

In this step, the notion of fairness comes into play. Here we can distinguish between [dataset metrics](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.BinaryLabelDatasetMetric.html#aif360.metrics.BinaryLabelDatasetMetric), which can be derived from a single dataset, and [classification metrics](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.ClassificationMetric.html), which take into account the differences between true outcomes and predictions.

First, let's look at the metrics of our original test data set including the true outcome.

In [ ]:
display(Markdown("## Original test dataset"))
show_dataset_metrics(dataset_orig_test)

In [ ]:
"""
Try to calculate the Statistical Parity Difference on your own by using the values 
given in the distribution tables above. Remember, gender=1 indicate female instances.

SPD = actually LTU women / all women - actually LTU men / all men

"""

act_ltu_women = 0
all_women = 1
act_ltu_men = 0
all_men = 1

print(act_ltu_women/all_women-act_ltu_men/all_men)

Let us now look at the classification metrics after predicting the outcome with the trained model. 

In [ ]:
display(Markdown("## Predicted test dataset"))
show_classification_metrics(dataset_orig_test, dataset_orig_test_pred)

In [ ]:
"""
Try to calculate the Equal Opportunity Difference on your own by using the values 
given in the confusion matrices above. Remember, gender=1 indicate female instances.

EOD = True Positive women / all actual positive women - True Positive men / all actual positive men

"""

tp_women = 0
all_pos_women = 1
tp_men = 0
all_pos_men = 1

print(tp_women/all_pos_women-tp_men/all_pos_men)

At the beginning, we set the classification threshold to 0.5, which means that every instance where the model predicts ltue=1 with a probability greater than 0.5 is assigned ltue_pred=1. In the following cell, we plot the F1 score and the fairness metrics across different thresholds to gain insight into how much the threshold affects fairness.

In [ ]:
thresh_arr = np.linspace(0.01, 0.99, 99)

get_fairness_over_thresholds(dataset_orig_test, lmod, thresh_arr, unprivileged_groups, privileged_groups, threshold=threshold)

## Removing the sensitive attribute

If the algorithm is found to make its decision depending on a protected attribute, it appears obvious to simply remove this feature from the dataset to correct for the bias. So, let's omit the variable gender during training and look at the fairness metrics.

In [ ]:
# Remove the variable gender from the datasets and convert them to BinaryLabelDatasets again

df_train = dataset_orig_train.convert_to_dataframe()[0]
df_train_ng = df_train.drop([sens_attr], axis=1)

df_test = dataset_orig_test.convert_to_dataframe()[0]
df_test_ng = df_test.drop([sens_attr], axis=1)

dataset_train_ng = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_train_ng,
    label_names=[target],
    protected_attribute_names=[]
)

dataset_test_ng = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=df_test_ng,
    label_names=[target],
    protected_attribute_names=[]
)

In [ ]:
# Train a Logistic Regression on the data without gender

lmod_ng = LogisticRegression(random_state=seed, solver="liblinear")
lmod_ng.fit(dataset_train_ng.features, dataset_train_ng.labels.ravel())

In [ ]:
# Predict the test data set with the new model

dataset_test_ng_pred = get_dataset_with_predictions(dataset_test_ng, lmod_ng, threshold)

In [ ]:
display(Markdown(f"## Confustion matrix"))
show_conf_matrix(dataset_orig_test, dataset_test_ng_pred, per_group=False)

In [ ]:
display(Markdown(f"## Classification report"))
print(classification_report(dataset_test_ng.labels, dataset_test_ng_pred.labels))

In [ ]:
# To examine the differences between genders, we create a dataset with gender assigned the scores and labels we predicted previously

dataset_test_ng_pred_wg = dataset_orig_test.copy(deepcopy=True)
dataset_test_ng_pred_wg.scores = dataset_test_ng_pred.scores
dataset_test_ng_pred_wg.labels = dataset_test_ng_pred.labels

In [ ]:
dataset_test_ng_metric = ClassificationMetric(dataset_orig_test,
        dataset_test_ng_pred_wg,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)

display(Markdown("## Predicted test dataset without sensitive attribute"))
show_classification_metrics(dataset_orig_test, dataset_test_ng_pred_wg)

In [ ]:
get_fairness_over_thresholds(dataset_orig_test, lmod, thresh_arr, unprivileged_groups, privileged_groups, threshold=threshold, dataset_with_scores=dataset_test_ng_pred_wg)

# Add-on

## Bias mitigation

To demonstrate the use of bias mitigation algorithms, we apply reweighing to the data at hand. This algorithm assigns a weight to each instance with respect to its sens_attr value and its true outcome. A classifier is then trained with the new instance weights.

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_orig_train)

In [ ]:
display(Markdown("## Transformed training dataset"))
show_dataset_metrics(dataset_transf_train)

### Train classifier on transformed data

In [ ]:
lmod_transf = LogisticRegression(random_state=seed, solver="liblinear")
lmod_transf = lmod_transf.fit(dataset_transf_train.features, dataset_transf_train.labels.ravel(), sample_weight=dataset_transf_train.instance_weights)

In [ ]:
dataset_transf_test_pred = get_dataset_with_predictions(dataset_orig_test, lmod_transf, threshold)

In [ ]:
display(Markdown(f"## Confustion matrix"))
show_conf_matrix(dataset_orig_test, dataset_transf_test_pred, per_group=False)

In [ ]:
display(Markdown(f"## Classification report"))
print(classification_report(dataset_orig_test.labels, dataset_transf_test_pred.labels))

In [ ]:
display(Markdown("## Predicted transformed test dataset"))
show_classification_metrics(dataset_orig_test, dataset_transf_test_pred, f1=False)

In [ ]:
get_fairness_mitigation_over_thresholds(dataset_orig_test, dataset_orig_test_pred, dataset_transf_test_pred, thresh_arr, unprivileged_groups, privileged_groups, threshold)

## Applying a Random Forest model

Of course, it is possible to use a model other than logistic regression for this classification problem. The following cells show the use of a Random Forest classifier and the resulting fairness evaluation.

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
rf=RandomForestClassifier(n_estimators=100, random_state=seed)

#Train the model using the training sets y_pred=clf.predict(X_test)
rf.fit(dataset_orig_train.features, dataset_orig_train.labels.ravel())

In [ ]:
dataset_orig_test_pred_rf = get_dataset_with_predictions(dataset_orig_test, rf, threshold)

In [ ]:
display(Markdown(f"## Confustion matrix"))
show_conf_matrix(dataset_orig_test, dataset_orig_test_pred_rf, per_group=False)

In [ ]:
display(Markdown(f"## Classification report"))
print(classification_report(dataset_orig_test.labels, dataset_orig_test_pred_rf.labels))

In [ ]:
display(Markdown("## Predicted test dataset"))
show_classification_metrics(dataset_orig_test, dataset_orig_test_pred_rf)

In [ ]:
thresh_arr = np.linspace(0.01, 0.99, 99)

get_fairness_over_thresholds(dataset_orig_test, rf, thresh_arr, unprivileged_groups, privileged_groups, threshold=threshold)